<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr1/blob/main/Lyakhov%20Igor/setup4_of_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендательная система для профпатолога (INSPECTRUM CLINIC)

## Анализ датасета



In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, Adadelta, RMSprop
from tensorflow.keras.metrics import *
#from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
# Дополнительные инструменты
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import utils
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 

from google.colab import data_table
data_table.enable_dataframe_formatter()

import random 
import math 
import os 
import re 
%matplotlib inline

In [6]:

import gdown
gdown.download('https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx', None, quiet=True)

'%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx'

In [7]:
# Считывание и запись данных
data = pd.read_excel('%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx')

In [8]:
# Количество строк и столбцов
data.shape

(1982, 4587)

In [9]:
# Количество элементов в таблице
data.size

9091434

In [10]:
# количество непустых записей в каждом столбце
data.count()

ДокументПрохождениеМедосмотра                                             1982
Клиент                                                                    1982
КлиентДатаРождения                                                        1982
КлиентПол                                                                 1982
КлиентПолПредставление                                                    1982
                                                                          ... 
Стоматология2_ВП2_ЗначениеПредставление                                      1
Терапия2_ЗаключениеФОГК_Значение                                             1
Терапия2_ЗаключениеФОГК_ЗначениеПредставление                                1
Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение                    1
Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление       1
Length: 4587, dtype: int64

In [11]:
# тип данных каждого столбца
data.dtypes

ДокументПрохождениеМедосмотра                                                     object
Клиент                                                                            object
КлиентДатаРождения                                                        datetime64[ns]
КлиентПол                                                                         object
КлиентПолПредставление                                                            object
                                                                               ...      
Стоматология2_ВП2_ЗначениеПредставление                                           object
Терапия2_ЗаключениеФОГК_Значение                                                  object
Терапия2_ЗаключениеФОГК_ЗначениеПредставление                                     object
Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение                         object
Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление            object
Length: 4587, dtype: 

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982 entries, 0 to 1981
Columns: 4587 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(1253), object(3290)
memory usage: 69.3+ MB


In [13]:
# bool type columns
bool_cols = data.dtypes[data.dtypes == 'bool']
bool_cols_count = len(bool_cols)
print(f'Колоноки типа bool: {bool_cols_count}')
print('-'*50)
print(bool_cols)

Колоноки типа bool: 1
--------------------------------------------------
ПсихОсвидетельствование    bool
dtype: object


In [14]:
# datetime64[ns] type columns
datetime_cols = data.dtypes[data.dtypes == 'datetime64[ns]']
datetime_cols_count = len(datetime_cols)
print(f'Колоноки типа datetime: {datetime_cols_count}')
print('-'*50)
print(datetime_cols)

Колоноки типа datetime: 43
--------------------------------------------------
КлиентДатаРождения                                        datetime64[ns]
ДатаЗавершенияМедосмотра                                  datetime64[ns]
ЛабораторныеИсследования1_ДатаПриема                      datetime64[ns]
Рентгенология1_ДатаПриема                                 datetime64[ns]
Рентгенология2_ДатаПриема                                 datetime64[ns]
ПсихиатрияНаркология1_ДатаПриема                          datetime64[ns]
Офтальмология1_ДатаПриема                                 datetime64[ns]
Оториноларингология1_ДатаПриема                           datetime64[ns]
ХирургНеИсп1_ДатаПриема                                   datetime64[ns]
Гинекология1_ДатаПриема                                   datetime64[ns]
ФункциональнаяДиагностика1_ДатаПриема                     datetime64[ns]
Стоматология1_ДатаПриема                                  datetime64[ns]
Эндоскопия1_ДатаПриема                        

In [15]:
# float64 type columns
float_cols = data.dtypes[data.dtypes == 'float64']
float_cols_count = len(float_cols)
print(f'Колоноки типа float: {float_cols_count}')
print('-'*50)
print(float_cols)

Колоноки типа float: 1253
--------------------------------------------------
ЛабораторныеИсследования1_ХР_Рекомендация2Представление    float64
ЛабораторныеИсследования1_ХР_РекомендацияСтр               float64
Рентгенология1_ХР_Рекомендация1                            float64
Рентгенология1_ХР_Рекомендация1Представление               float64
Рентгенология1_ХР_Рекомендация2                            float64
                                                            ...   
Оториноларингология3_Классификация2Представление1          float64
Оториноларингология3_Классификация31                       float64
Оториноларингология3_Классификация3Представление1          float64
Оториноларингология3_Противопоказание1                     float64
Оториноларингология3_ПротивопоказаниеПредставление1        float64
Length: 1253, dtype: object


In [16]:
# object-string type columns
obj_cols = data.dtypes[data.dtypes == 'O']
obj_cols_count = len(obj_cols)
print(f'Колоноки типа object: {obj_cols_count}')
print('-'*50)
print(obj_cols)

Колоноки типа object: 3290
--------------------------------------------------
ДокументПрохождениеМедосмотра                                             object
Клиент                                                                    object
КлиентПол                                                                 object
КлиентПолПредставление                                                    object
ТипМедосмотра                                                             object
                                                                           ...  
Стоматология2_ВП2_ЗначениеПредставление                                   object
Терапия2_ЗаключениеФОГК_Значение                                          object
Терапия2_ЗаключениеФОГК_ЗначениеПредставление                             object
Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение                 object
Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление    object
Length: 3290, dtype: object


In [17]:
# число колонок
total_cols_count = bool_cols_count + datetime_cols_count + float_cols_count + obj_cols_count
print('Число колонок:')
print(f'  bool: {bool_cols_count}')
print(f'  datetime64[ns]: {datetime_cols_count}')
print(f'  float64: {float_cols_count}')
print(f'  object-string: {obj_cols_count}')
print('-'*50)
print(f'Общее число колонок: {total_cols_count}')

Число колонок:
  bool: 1
  datetime64[ns]: 43
  float64: 1253
  object-string: 3290
--------------------------------------------------
Общее число колонок: 4587


In [18]:
# названия колонок
col_names = list(data.columns)
col_names_count = len(col_names)
print(f'Число названий колонок: {col_names_count}')
print('-'*50)
print(col_names)

Число названий колонок: 4587
--------------------------------------------------
['ДокументПрохождениеМедосмотра', 'Клиент', 'КлиентДатаРождения', 'КлиентПол', 'КлиентПолПредставление', 'ДатаЗавершенияМедосмотра', 'ТипМедосмотра', 'ТипМедосмотраПредставление', 'ПсихОсвидетельствование', 'Контрагент', 'Цех', 'Профессия', 'ЗаключениеМК', 'ЗаключениеМКПредставление', 'Заключение', 'ДиспансернаяГруппа', 'ДиспансернаяГруппаПредставление', 'ВредныеФакторы', 'ЛабораторныеИсследования1_ДокументПрием', 'ЛабораторныеИсследования1_ДатаПриема', 'ЛабораторныеИсследования1_Салон', 'ЛабораторныеИсследования1_СалонПредставление', 'ЛабораторныеИсследования1_Специализация', 'ЛабораторныеИсследования1_СпециализацияПредставление', 'ЛабораторныеИсследования1_ХР_Рекомендация1', 'ЛабораторныеИсследования1_ХР_Рекомендация1Представление', 'ЛабораторныеИсследования1_ХР_Рекомендация2', 'ЛабораторныеИсследования1_ХР_Рекомендация2Представление', 'ЛабораторныеИсследования1_ХР_РекомендацияСтр', 'ЛабораторныеИсследова

In [19]:
# количество пропусков в каждом столбце
data.isna().sum()

ДокументПрохождениеМедосмотра                                                0
Клиент                                                                       0
КлиентДатаРождения                                                           0
КлиентПол                                                                    0
КлиентПолПредставление                                                       0
                                                                          ... 
Стоматология2_ВП2_ЗначениеПредставление                                   1981
Терапия2_ЗаключениеФОГК_Значение                                          1981
Терапия2_ЗаключениеФОГК_ЗначениеПредставление                             1981
Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение                 1981
Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление    1981
Length: 4587, dtype: int64

In [20]:
data.head()

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
0,7f41ca7e-7dac-11ec-80fc-0cc47aab8067,c91e3407-b81f-11e9-80cf-0cc47aab8067,1976-10-24,Женский,Женский,2022-01-25,Периодический,Периодический,False,23c173a7-30d9-11e7-80d8-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2ee1f9ac-83dd-11ec-80fc-0cc47aab8067,1a1a5fea-8b7f-11eb-80e8-0cc47aab8067,1987-11-18,Мужской,Мужской,2022-02-02,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9cbb191e-8e24-11ec-80fe-0cc47aab8067,7fe324e0-fa51-11eb-80f6-0cc47aab8067,1985-09-08,Мужской,Мужской,2022-02-15,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b8a33910-8eec-11ec-80fe-0cc47aab8067,98f9d761-8d65-11ec-80fe-0cc47aab8067,1988-04-25,Мужской,Мужской,2022-02-16,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0bb0d181-938c-11ec-80fe-0cc47aab8067,009aac7d-7bcf-11eb-80e8-0cc47aab8067,1984-04-17,Мужской,Мужской,2022-02-22,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# долю пропусков от общего количества элементов
data.isna().sum() / data.shape[0]

ДокументПрохождениеМедосмотра                                             0.000000
Клиент                                                                    0.000000
КлиентДатаРождения                                                        0.000000
КлиентПол                                                                 0.000000
КлиентПолПредставление                                                    0.000000
                                                                            ...   
Стоматология2_ВП2_ЗначениеПредставление                                   0.999495
Терапия2_ЗаключениеФОГК_Значение                                          0.999495
Терапия2_ЗаключениеФОГК_ЗначениеПредставление                             0.999495
Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение                 0.999495
Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление    0.999495
Length: 4587, dtype: float64

In [22]:
# общее количество пропусков во всей таблице
data.isna().sum().sum()

8106978

In [23]:
# общее количество пропусков во всей таблице в %
proc = data.isna().sum().sum() / data.size
print('{:.1%}'.format(proc))

89.2%


In [24]:
# Для дальнешей работы делаем копию таблицы
data_backup = data.copy()
df = data.copy()

In [25]:
# проанализируем целевую колонку на уникальные значения
unique_names = df['ЗаключениеМК'].unique().tolist()
print(unique_names)

['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах', 'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден', 'ОграниченноГоден', nan, 'машинист бульдозера', 'Газоспасатель', ' Газоспасатель']


In [26]:
# удаляем строки с очевидными ошибками

In [27]:
# удаляем nan, 'машинист бульдозера', 'Газоспасатель', ' Газоспасатель'
df = df.drop(df[(df['ЗаключениеМК'].isnull()) | (df['ЗаключениеМК'] == 'машинист бульдозера') | (df['ЗаключениеМК'] == 'Газоспасатель') | (df['ЗаключениеМК'] == ' Газоспасатель') ].index)


In [28]:
#проверяем список уникальных значений
unique_names2 = df['ЗаключениеМК'].unique().tolist()
print(unique_names2)

['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах', 'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден', 'ОграниченноГоден']


In [29]:
diagnoz = df['ЗаключениеМК']
diagnoz

0                        Годен
1                        Годен
2       ГоденСКоррекциейЗрения
3                        Годен
4       ГоденСКоррекциейЗрения
                 ...          
1961                     Годен
1962                     Годен
1972                     Годен
1973                     Годен
1981                     Годен
Name: ЗаключениеМК, Length: 1571, dtype: object

In [30]:
diagnoz.value_counts()

Годен                               1434
ГоденСКоррекциейЗрения               123
ВременноНегоден                        9
НуждаетсяВДообследованииИЛечении       2
ОграниченноГоден                       2
ГоденБезРаботНаВысотах                 1
Name: ЗаключениеМК, dtype: int64

In [31]:
# Приведение к 3 классам методом map()
new_val = {'ГоденСКоррекциейЗрения':'ОграниченноГоден','ГоденБезРаботНаВысотах':'ОграниченноГоден','НуждаетсяВДообследованииИЛечении':'НеГоден','ВременноНегоден':'НеГоден','Годен':'Годен','ОграниченноГоден':'ОграниченноГоден'}
diagnoz = diagnoz.map(new_val)

In [32]:
diagnoz.value_counts()

Годен               1434
ОграниченноГоден     126
НеГоден               11
Name: ЗаключениеМК, dtype: int64

In [34]:
#Удалим колонки не имеющие записей
print(f'Размер до - {df.shape}')
df_c = df.dropna(axis=1, how='all')
print(f'Размер после - {df_cor.shape}')

Размер до - (1571, 4587)
Размер после - (1571, 3578)


In [35]:
# создадим список колонок датафрейма.
cols_list = df_c.columns.tolist()
len(cols_list)

3578

In [36]:
cols_list

['ДокументПрохождениеМедосмотра',
 'Клиент',
 'КлиентДатаРождения',
 'КлиентПол',
 'КлиентПолПредставление',
 'ДатаЗавершенияМедосмотра',
 'ТипМедосмотра',
 'ТипМедосмотраПредставление',
 'ПсихОсвидетельствование',
 'Контрагент',
 'Цех',
 'Профессия',
 'ЗаключениеМК',
 'ЗаключениеМКПредставление',
 'Заключение',
 'ДиспансернаяГруппа',
 'ДиспансернаяГруппаПредставление',
 'ВредныеФакторы',
 'ЛабораторныеИсследования1_ДокументПрием',
 'ЛабораторныеИсследования1_ДатаПриема',
 'ЛабораторныеИсследования1_Салон',
 'ЛабораторныеИсследования1_СалонПредставление',
 'ЛабораторныеИсследования1_Специализация',
 'ЛабораторныеИсследования1_СпециализацияПредставление',
 'ЛабораторныеИсследования1_ОАК_Значение',
 'ЛабораторныеИсследования1_ОАК_ЗначениеПредставление',
 'ЛабораторныеИсследования1_Биохимия_Значение',
 'ЛабораторныеИсследования1_Биохимия_ЗначениеПредставление',
 'ЛабораторныеИсследования1_ОАМ_Значение',
 'ЛабораторныеИсследования1_ОАМ_ЗначениеПредставление',
 'ЛабораторныеИсследования1_Ма

In [37]:
# Составим список учитываемых колонок
selected_col = ['КлиентДатаРождения','ДатаЗавершенияМедосмотра','КлиентПол','Профессия','ЗаключениеМК','ДиспансернаяГруппа',
           'ВредныеФакторы','Терапия1_П_Значение','Терапия1_ЧССТерапевт_Значение']

In [38]:

# Добавим некоторые колонки по условию
for elem in cols_list:
    if ('Рекомендация' in elem and 'Представление' in elem) or 'РекомендацияСтр' in elem:       # Добавим колони с рекомендациями
        selected_col.append(elem)
        continue
    if 'МКБ' in elem or 'Детализация' in elem:                  # Колонки с кодами МКБ и Детализация
        selected_col.append(elem)
        continue
    if 'Противопоказание' in elem and 'Представление' in elem:   # Противопоказания 
        selected_col.append(elem)
        continue
    if 'Профнепригодность' in elem and 'Представление' in elem:  # Профнепригодность
        selected_col.append(elem)
        continue
    if 'Диагноз' in elem and 'Представление' in elem:            # Диагнозы
        selected_col.append(elem)
        continue
    if 'ГоденНегоден' in elem and 'Представление' in elem:      # ГоденНегоден
        selected_col.append(elem)
        continue
    if 'Анамнез' in elem and 'Представление' in elem:           # Анамнез
        selected_col.append(elem)
        continue
    if 'Аудиометрия' in elem and 'Представление' in elem:       # Аудиометрия
        selected_col.append(elem)
        continue
    if 'Заключение_' in elem and 'Представление' in elem:       # Заключения по специалистам если такие есть.   
        selected_col.append(elem)
        continue
# колонки содержащие цировые данные (могут иметь тип не floan и int - преобразовывать будем потом)        
    if 'АртериальноеДавление' in elem and 'Представление' in elem:    #      
        selected_col.append(elem)
        continue
    if 'Температура' in elem and 'Представление' in elem:          #
        selected_col.append(elem)
        continue
    if 'Сатурация' in elem and 'Представление' in elem:          #
        selected_col.append(elem)
        continue
    if 'ИМТ' in elem and 'Представление' in elem:          #
        selected_col.append(elem)
        continue

print(len(selected_col))
print(selected_col[:5])
     

481
['КлиентДатаРождения', 'ДатаЗавершенияМедосмотра', 'КлиентПол', 'Профессия', 'ЗаключениеМК']


In [39]:
# Сократим датафрейм до отобранных колонок.
df_c = df_c[selected_col]
df_c.shape
     

(1571, 481)

In [40]:
# Проверка результата: создадим таблицу содержащую данные по заполненности колонки, количеству уникальных значений, и типу данных
counts = df_c.count(axis=0)
uniques = df_c.nunique(axis=0)
types = df_c.dtypes
df_new = pd.concat([counts.to_frame(name='count'), uniques.to_frame(name='unique'), types.to_frame(name='type')], axis= 1 )
df_new

,count,unique,type
КлиентДатаРождения,1571,1426,datetime64[ns]
ДатаЗавершенияМедосмотра,1571,84,datetime64[ns]
КлиентПол,1571,2,object
Профессия,1571,557,object
ЗаключениеМК,1571,6,object
...,...,...,...
Оториноларингология3_ПрофнепригодностьВременнаяПредставление1,1,1,object
Оториноларингология3_ГоденНегоден_ЗначениеПредставление,1,1,object
Оториноларингология3_Анамнез_ЗначениеПредставление,1,1,object
Оториноларингология3_Диагноз_ЗначениеПредставление,1,1,object


In [41]:
# Удалим колонки с одной записью 
del_c = list(df_new[(df_new['count'] == 1)].index)
print(len(del_c))
df_c = df_c.drop(del_c, axis=1)
df_c.shape

30


(1571, 451)

In [42]:
df_c.shape

(1571, 451)

In [43]:
cols_list = df_c.columns.tolist()
len(cols_list)

451

In [47]:
# Объединение родственных колонок по специалистам, используя написанную вначале процедуру col_concat()
# с последующим удалением использованных колонок.
lst_par1 = ['Терап','Хирург','Гинеколог','Стоматолог','Дермато','Психиатр','Невролог','Офтальмолог','Оториноларинголог',
            'Рентгенолог','УЗИ','Лаборатор','Процедур','Эндоскоп','Функционал']
lst_par2 = ['Рекомендация','МКБ','ИМТ','Детализация','Профнепригодность','ГоденНегоден','Диагноз','Анамнез','Аудиометрия',
            'АртериальноеДавление_','Температура','Сатурация','АртериальноеДавление21']
for e1 in lst_par1:
    cols1_list = df_c.columns.tolist()  # Пересоздаем список колонок чтобы исключить удаленные
    tst_list = []
    for e in cols1_list:
        if e1 in e:
            tst_list.append(e)
    for e2 in lst_par2:
        conc_list = []
        for e in tst_list:
            if e2 in e:
                conc_list.append(e)
        if len(conc_list) > 1:
            row_con = conc_list.pop(0)
            col_concat(row_con, conc_list) 
            df_c = df_c.drop(conc_list, axis=1) 

print(df_c.shape)
print(df_c)    

(1571, 98)
     КлиентДатаРождения ДатаЗавершенияМедосмотра КлиентПол  \
0            1976-10-24               2022-01-25   Женский   
1            1987-11-18               2022-02-02   Мужской   
2            1985-09-08               2022-02-15   Мужской   
3            1988-04-25               2022-02-16   Мужской   
4            1984-04-17               2022-02-22   Мужской   
...                 ...                      ...       ...   
1961         1993-12-29               2022-11-30   Мужской   
1962         1989-07-10               2022-11-30   Мужской   
1972         1987-02-04               2022-11-30   Мужской   
1973         1993-02-04               2022-11-30   Мужской   
1981         1971-09-14               2022-12-02   Мужской   

                                              Профессия  \
0             Ведущий специалист по организации обуения   
1                                    Ведущий специалист   
2                                      Главный менеджер   
3       

In [48]:
# процет незаполненных ячеек
df_c.isna().sum().sum() * 100 / df_c.size

59.736421621481185

In [49]:
# Приведем в порядок цифровые колонки 
map_x = lambda x: x if len(str(x)) <= 3 else str(x)[:3]
df_c['Терапия1_АртериальноеДавление_ЗначениеПредставление'] = df_c['Терапия1_АртериальноеДавление_ЗначениеПредставление'].map(map_x)
df_c['Терапия1_АртериальноеДавление21_ЗначениеПредставление'] = df_c['Терапия1_АртериальноеДавление21_ЗначениеПредставление'].map(map_x)
# И переведем её в float формат
df_c = df_c.astype({'Терапия1_АртериальноеДавление_ЗначениеПредставление': np.float64})
df_c = df_c.astype({'Терапия1_АртериальноеДавление21_ЗначениеПредставление': np.float64})
df_c.loc[df_c['Терапия1_АртериальноеДавление21_ЗначениеПредставление'] > 300,['Терапия1_АртериальноеДавление21_ЗначениеПредставление']] = df_c['Терапия1_АртериальноеДавление21_ЗначениеПредставление']/10
map_y = lambda x: str(x).replace(',','.').replace('/','.').replace('..','.')
df_c['Терапия1_Температура_ЗначениеПредставление'] = df_c['Терапия1_Температура_ЗначениеПредставление'].map(map_y)
df_c = df_c.astype({'Терапия1_Температура_ЗначениеПредставление': np.float64})
df_c = df_c.astype({'Терапия1_Сатурация_ЗначениеПредставление': np.float64})
df_c['Терапия1_ИМТ_ЗначениеПредставление'] = df_c['Терапия1_ИМТ_ЗначениеПредставление'].map(map_y)
df_c = df_c.astype({'Терапия1_ИМТ_ЗначениеПредставление': np.float64})
print(df_c['Терапия1_АртериальноеДавление21_ЗначениеПредставление'].unique())
print(df_c['Терапия1_АртериальноеДавление_ЗначениеПредставление'].unique())
print(df_c['Терапия1_Температура_ЗначениеПредставление'].unique())

[  nan  82.   80.   60.   70.   90.  110.  120.   75.   85.   84.   77.
  72.   68.   78.   83.   64.   76.   74.   86.   89.   87.   71.   88.
 112.   92.   66.  100.   62.   81.   80.1  99.   91.   97.  111.  104.
 102.   95.   98.   69.   96.  105.   73.  108.   63.  101.   93.   79.
  67.  107.  106.   94.  103.  109.   80.5  65.   61.   20.   58.   40.
  59. ]
[140. 150. 120. 110. 105. 130. 125. 133. 100. 160.  nan 180. 115.  90.
 135. 128. 138. 112. 129. 118. 155. 102. 144. 107. 141. 182. 145. 131.
 123. 193. 121. 127. 113. 132. 109. 117. 137. 124. 116. 171. 151. 142.
 134. 146. 168. 119. 111. 177. 165. 122. 153. 159. 143. 147. 148. 167.
 136. 139. 189. 166. 162. 158. 172. 169. 152. 126. 198. 161. 174. 175.
 186. 108. 106. 170. 154. 149. 173. 157. 114. 163.  95. 101.]
[36.4 36.5 36.6 36.3 36.2  nan]


In [51]:
# Проверка результата пересоберем df_new
counts = df_c.count(axis=0)
uniques = df_c.nunique(axis=0)
types = df_c.dtypes
df_new = pd.concat([counts.to_frame(name='count'), uniques.to_frame(name='unique'), types.to_frame(name='type')], axis= 1 )
df_new

,count,unique,type
КлиентДатаРождения,1571,1426,datetime64[ns]
ДатаЗавершенияМедосмотра,1571,84,datetime64[ns]
КлиентПол,1571,2,object
Профессия,1571,557,object
ЗаключениеМК,1571,6,object
...,...,...,...
ФункциональнаяДиагностика1_ДиагнозПредставление1,6,1,object
ФункциональнаяДиагностика1_ПрофнепригодностьВременнаяПредставление1,6,1,object
Эндоскопия1_МКБ101,4,2,object
Эндоскопия1_ДиагнозПредставление1,4,2,object


In [55]:
# ВредныеФакторы
factors_list = []
for elem in data['ВредныеФакторы']:
  factors_list += elem.split(',')
factors_list = list(set(factors_list))
factors_list

['1.8.1',
 '11',
 '27',
 '15',
 '1.14.1',
 '1.23',
 '24',
 '11.1',
 '1.8.3.2',
 '8',
 '4.2.3',
 '1.39',
 '1.8.2',
 '1.8.3.1',
 '3.1.8.1',
 '1.3',
 '4.2.4',
 '1.29.2',
 '3.1.8.3',
 '4.1',
 '3.1.8.2',
 '1.36',
 '25',
 '4.2.1',
 '18.1',
 '1.2',
 '11.4',
 '1.49.2',
 '4.4',
 '12',
 '4.2.5',
 '1.17',
 '1.37.1.2',
 '1.37.1',
 '3.4',
 '1.37',
 '6.2',
 '1.45.1',
 'IIIа',
 '11.2',
 '1.50',
 '3.1.7',
 '17',
 '5.2',
 '1.48',
 '1.36.1',
 '1.8.2.1',
 '1.34.1',
 '1.15',
 '6.1',
 '4.3.2',
 '1.49.14',
 '1.29.4',
 '3.1',
 'Годен к работе в указанных условиях',
 '1.8.2.2',
 '1.29',
 '14',
 '1.1',
 '1.14.2',
 '4.2',
 '4.5',
 '4.8',
 '1.8.1.1',
 '1.8.1.2',
 '22',
 '1.29.1',
 '1.30.1',
 '2.4.3',
 '13',
 '1.24',
 '1.8.3',
 '18.2',
 '3.1.2',
 '9',
 '11.3',
 '5.2.1',
 '10',
 '5.1',
 '1.37.1.1',
 '1.19.1',
 'II',
 '1.18.3',
 '2.4',
 '6',
 '4.7',
 '26',
 '4.3.1',
 '23',
 '18',
 '4.9',
 '1.49',
 '1.46',
 '4.10',
 '4.6']

In [60]:
factors_list = sorted([x for x in factors_list if len(re.findall(r'\d',x))>0], key = lambda x: [int(s) for s in x.split('.')])
print(len(factors_list))
factors_list

92


['1.1',
 '1.2',
 '1.3',
 '1.8.1',
 '1.8.1.1',
 '1.8.1.2',
 '1.8.2',
 '1.8.2.1',
 '1.8.2.2',
 '1.8.3',
 '1.8.3.1',
 '1.8.3.2',
 '1.14.1',
 '1.14.2',
 '1.15',
 '1.17',
 '1.18.3',
 '1.19.1',
 '1.23',
 '1.24',
 '1.29',
 '1.29.1',
 '1.29.2',
 '1.29.4',
 '1.30.1',
 '1.34.1',
 '1.36',
 '1.36.1',
 '1.37',
 '1.37.1',
 '1.37.1.1',
 '1.37.1.2',
 '1.39',
 '1.45.1',
 '1.46',
 '1.48',
 '1.49',
 '1.49.2',
 '1.49.14',
 '1.50',
 '2.4',
 '2.4.3',
 '3.1',
 '3.1.2',
 '3.1.7',
 '3.1.8.1',
 '3.1.8.2',
 '3.1.8.3',
 '3.4',
 '4.1',
 '4.2',
 '4.2.1',
 '4.2.3',
 '4.2.4',
 '4.2.5',
 '4.3.1',
 '4.3.2',
 '4.4',
 '4.5',
 '4.6',
 '4.7',
 '4.8',
 '4.9',
 '4.10',
 '5.1',
 '5.2',
 '5.2.1',
 '6',
 '6.1',
 '6.2',
 '8',
 '9',
 '10',
 '11',
 '11.1',
 '11.2',
 '11.3',
 '11.4',
 '12',
 '13',
 '14',
 '15',
 '17',
 '18',
 '18.1',
 '18.2',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27']

In [62]:
X_adverse_factors = pd.DataFrame({'adverse_factor_' + elem: data['ВредныеФакторы'].apply(lambda x: 1 if elem in x.split(',') else 0) for elem in factors_list}, index = data.index)
print(X_adverse_factors.shape)
X_adverse_factors

(1982, 92)


,adverse_factor_1.1,adverse_factor_1.2,adverse_factor_1.3,adverse_factor_1.8.1,adverse_factor_1.8.1.1,adverse_factor_1.8.1.2,adverse_factor_1.8.2,adverse_factor_1.8.2.1,adverse_factor_1.8.2.2,adverse_factor_1.8.3,...,adverse_factor_17,adverse_factor_18,adverse_factor_18.1,adverse_factor_18.2,adverse_factor_22,adverse_factor_23,adverse_factor_24,adverse_factor_25,adverse_factor_26,adverse_factor_27
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1978,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1979,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [63]:
X_adverse_factors.sum(axis=1)[X_adverse_factors.sum(axis=1)==0]

843     0
851     0
1022    0
1024    0
dtype: int64

In [64]:
X_adverse_factors.sum().sort_values()

adverse_factor_1.8.3      1
adverse_factor_2.4        1
adverse_factor_2.4.3      1
adverse_factor_1.36       1
adverse_factor_1.46       1
                       ... 
adverse_factor_5.1      351
adverse_factor_6.1      351
adverse_factor_4.2.5    356
adverse_factor_4.4      432
adverse_factor_11.1     523
Length: 92, dtype: int64

In [65]:
adverse_factors_aux_info = pd.DataFrame(X_adverse_factors.sum(), columns = ['n_occurences'])
adverse_factors_aux_info.sort_values('n_occurences')

,n_occurences
adverse_factor_1.8.3,1
adverse_factor_2.4,1
adverse_factor_2.4.3,1
adverse_factor_1.36,1
adverse_factor_1.46,1
...,...
adverse_factor_5.1,351
adverse_factor_6.1,351
adverse_factor_4.2.5,356
adverse_factor_4.4,432


In [59]:
profy_list = []
for elem in data['Профессия']:
  profy_list += elem.split(',')
profy_list = list(set(profy_list))
profy_list

['Слесарь по КИПиА 5 разряда',
 'Электромонтажник по силовым сетям и электрооборудованию ',
 'Оператор станка',
 'Электрик',
 'Машинист ЦА',
 'Водитель снегохода 3 разряда',
 'Машинист копра УГМК',
 'Воспитатель',
 'Упаковщик',
 'Взрывник 6 раазряда',
 'Водитель по доставке готовой продукции',
 'Монтажник технологических трубопроводов ',
 'Монтажник по монтажу стальных и железобетонных конструкций ',
 'Машинист буровой установки',
 'старший продавец',
 'Помощник бурильщика капитального ремонта скважин',
 'Главный агроном',
 'Слесарь-электрик',
 'Водитель автомобиля (трала)',
 'Рабочий на геофизических работах 2 разряда',
 'Главный специалист ',
 'Геофизик 3 категории',
 'Горничная',
 'Инженер - проектировщик 1 категории',
 'Уборщик служебных и производственных помещений',
 'Начальник районного отдела',
 'Машинист бульдозера',
 'Ведущий специалист по организации обуения',
 'Ведущий специалист',
 'Начальник склада СГП',
 'Инженер по эксплуатации и качеству',
 'Старший  инженер',
 'Террит

In [74]:
# Проведем колонку "Профессия" через токенайзер отбросим числа, чтобы не учитывать разряды и номера категорий, а только сами специальности.
words_prof = 508
tokenizer_spec = Tokenizer(num_words=words_prof, filters='i1234567890!"#$%&()*+,-–—./…:;<=>?@[\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)
tokenizer_spec.fit_on_texts(df_c['Профессия'].tolist())
max(tokenizer_spec.word_index.values())

508

In [ ]:
nan_lst = df1[df1['ТипМедосмотра'].isnull()].index
print(f'"ТипМедосмотра": число строк с пустыми значениями: {len(nan_lst)}')
u_lst = df1['ТипМедосмотра'].unique().tolist()
print(f'{len(u_lst)}: {u_lst}')